In [1]:
import cv2
import pytesseract

Load the image

In [2]:
img = cv2.imread('text.png')

Convert to grayscale

In [3]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Apply threshold

In [4]:
thresh = cv2.threshold(gray, 150,255, cv2.THRESH_BINARY_INV)[1]

OCR

In [7]:
text = pytesseract.image_to_string(thresh)
print("Extracted Text:", text)

Extracted Text: Q11. How does prompt engineering influence the output of LLMs?

Ans - Prompt engineering involves crafting input prompts to guide an
LLM’s output effectively. Since LLMs are highly sensitive to input
phrasing, a well-designed prompt can significantly influence the
quality and relevance of the response. For example, adding context
or specific instructions within the prompt can improve accuracy in
tasks like summarization or question-answering. Prompt engineering
is especially useful in zero-shot and few-shot learning scenarios,
where task-specific examples are minimal.

The Cycle of Prompt Engineering

Design Prampt

Refine Prompt Submit to LLM

wo Foc Output
Output



Automatic Document Scanner

In [20]:
import cv2
import imutils
import numpy as np


In [34]:
# Load and resize image
image = cv2.imread("paper.jpeg")
if image is None:
    print("Error: Image not loaded. Check file path.")
    exit()
orig = image.copy()
image = imutils.resize(image, height=600) 


In [35]:
# Preprocess
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)
# thresholding for uneven lighting
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                              cv2.THRESH_BINARY_INV, 11, 2)
edged = cv2.Canny(thresh, 50, 150)  # Adjusted Canny thresholds
cv2.imshow("Edged", edged)
cv2.waitKey(0)


-1

In [36]:
# Find contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]  # Top 10 contours
print(f"Number of contours: {len(cnts)}")


Number of contours: 10


In [37]:

# Detect document contour
doc_cnt = None
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        doc_cnt = approx
        cv2.drawContours(image, [doc_cnt], -1, (0, 255, 0), 2)
        cv2.imshow("Detected Contour", image)
        cv2.waitKey(0)
        break

if doc_cnt is None:
    print("No quadrilateral detected. Adjust image or parameters.")
    exit()


In [38]:

# Perspective transform
warped = four_point_transform(orig, doc_cnt.reshape(4, 2))

# Resize for display and save
scanned = imutils.resize(warped, height=800)
cv2.imshow("Scanned", scanned)
cv2.waitKey(0)
cv2.imwrite("scanned_document.jpg", scanned)  # Save the output
cv2.destroyAllWindows()

In [26]:
# Helper functions
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect


In [27]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped